
### 5. 텐서보드와 모델 재사용

#### 5.1 학습 모델 저장 및 재사용

앞 장의 데이터를 CSV 파일로 분리한 뒤 해당 파일을 읽어 들여 사용해본다.  

데이터를 data.csv 파일로 저장한다. 한글이 있으면 문제가 생길 수 있음.  


--------
데이터를 읽어 들이고 변환한다.


In [1]:
import tensorflow as tf
import numpy as np

data= np.loadtxt('.\data\data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


loadtxt 함수의 nupack 매개변수, transpose 함수의 데이터 구조 변환   

![데이터 변환](.\img\5\img1.JPG)


읽어 들이거나 잘라낸 데이터의 행과 열을 뒤바꿔주는 옵션과 함수이다.  
딥러닝에서는 행과 열을 상황에 맞게 변환하는 일이 잦다.  

-------------
신경망 모델 구성  


In [2]:

# 모델을 저장할 때 쓸 변수 생성
# 직접 사용되지는 않고, 학습 횟수를 카운트 한다.
global_step = tf.Variable(0, trainable=False, name='global_step')


----
앞 장보다 계층을 하나 더 늘리고, 편향 없이 가중치만 사용한다.

In [3]:

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))    # 편향 없음

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.nn.relu(tf.matmul(L2, W3))


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
# global_step 변수를 넘겨주면 최적화 함수가 학습용 변수들을 최적화 할 때마다 해당 변수 값을 1씩 증가시킨다.
train_op = optimizer.minimize(cost, global_step=global_step)


**!** 신경망의 계층 수와 은닉층 뉴런 수(예제 [10, 20] 부분)를 늘리면 복잡도가 높은 문제를 해결하는데 도움됨.  
**하지만 무조건 항상 도움 되지는 않음** -> **과적합** 문제 발생 가능  
모델 구성에서 계층과 뉴런 수를 최적화 하는 것이 중요!!  


----
세션을 열고 최적화를 실행한다.  
모델을 불러들이고 저장하는 코드


In [4]:
sess = tf.Session()

# global+variables() : 앞서 정의한 변수들을 가져오는 함수. 
# 정의한 변수를 모두 가져와서 이 변수들을 파일에 저장하거나 이전 학습한 결과를 불러와 담는 변수들로 사용한다.
saver = tf.train.Saver(tf.global_variables())


----
./model 디렉코리에 기존에 학습해둔 모델이 있는지 확인한다.  
모델이 있으면 saver.restore 함수로 불러오고, 아니면 변수를 새로 초기화한다.   
학습된 모델을 저장한 파일을 **체크포인트 파일**이라고 한다.

In [5]:

ckpt = tf.train.get_checkpoint_state('.\model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from .\model\dnn.ckpt-12



-------------
최적화를 수행한다.  
앞 장과 달리 step -> global_step(텐서 타입 변수) 값으로 학습 횟수를 출력한다.

In [8]:
# 재 학습 결과를 알기위해 2번 학습한다.
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d' % sess.run(global_step),
         'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤 학습된 변수들을 지정한 체크포인트 파일에 저장
# global_step 값은 저장할 파일 이름에 추가로 붙는다. 텐서 변수 또는 숫자를 넣을 수 있다.
# 이를 이용해 여러 체크포인트를 만들 수 있다.
saver.save(sess, '.\model\dnn.ckpt', global_step=global_step)


# 예측 결과와 정확도를 확인한다.
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

print('예측값::', sess.run(prediction, feed_dict={X: x_data}))
print('실제값::', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


Step: 17 Cost: 1.117
Step: 18 Cost: 1.107
예측값:: [0 0 2 0 0 1]
실제값:: [0 1 2 0 0 2]
정확도:: 66.67



첫 번째 결과  
Step: 1 Cost: 1.360  
Step: 2 Cost: 1.279  
예측값:: [0 1 1 0 0 0]  
실제값:: [0 1 2 0 0 2]  
정확도:: 66.67  


두 번째 결과  
Step: 3 Cost: 1.211  
Step: 4 Cost: 1.150  
예측값:: [0 1 1 0 0 0]  
실제값:: [0 1 2 0 0 2]  
정확도:: 66.67  

총 2번 새로 시작했지만 global_step으로 저장한 값을 불러와서 증가시켜 3부터 시작한다. 


#### 5.2 텐서보드 사용하기


In [1]:
import tensorflow as tf
import numpy as np

data= np.loadtxt('.\data\data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=True, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [2]:
# 신경망의 각 계층에 다음 코드를 덧붙여준다.
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))


with tf.name_scope 로 묶은 블록은 텐서보드에서 한 계층 내부를 표현해준다.  
name='W1'으로 이름붙이면 텐서보드에서 쉽게 확인 가능.  

*플레이스홀더, 연산, 함수 등 모든 텐서에 붙일 수 있다.*  


In [3]:
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.nn.relu(tf.matmul(W3, L2))

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    # 손실값을 추적하기 위해 수직할 값 지정
    tf.summary.scalar('cost', cost)

tf.summary.scalar : 값이 하나인 텐서를 수집할 때 사용  

----------
모델을 불러들이거나 초기화 한다.

In [4]:
tf.reset_default_graph()

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('.\model')

if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else :
    sess.run(tf.global_variables_initializer())
    
# 위에서 지정한 텐서들을 수집한다.
maerged = tf.summary.merge_all()

# 그래프와 텐서들의 값을 저장할 디렉토리를 설정한다.
writer = tf.summary.FileWriter('.\logs', sess.graph)

ValueError: No variables to save